# Explore here

In [2]:
import os
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv

# load the .env file variables
load_dotenv()



ModuleNotFoundError: No module named 'seaborn'

In [1]:
# Step 2: Initial config
!pip install spotipy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# Step 3: Environment variables
from dotenv import load_dotenv
load_dotenv()

import os

client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

In [6]:
# Step 4: Obtain access token
import base64
import requests

# Encode the client credentials
client_credentials = f"{client_id}:{client_secret}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Prepare the token request
token_url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {client_credentials_base64.decode()}"
}
data = {
    "grant_type": "client_credentials"
}

# Request access token
response = requests.post(token_url, headers=headers, data=data)

# Check if the request was successful
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained!")
else:
    print(f"Failed to obtain access token. Status code: {response.status_code}")
    print(response.text)

Access token obtained!


In [7]:
# Step 5: Search for an artist
# Set the access token in the headers
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Define the endpoint and parameters
search_url = "https://api.spotify.com/v1/search"
params = {
    "q": "Rainbow Kitten Surprise",  # Artist name to search for
    "type": "artist",
    "limit": 1
}

# Make the API request to search for the artist
response = requests.get(search_url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    response_data = response.json()
    print(f"Response Data: {response_data}")
    artist_id = response_data['artists']['items'][0]['id']
    artist_name = response_data['artists']['items'][0]['name']
    # print(f"Found artist: {artist_name} (ID: {artist_id})")
else:
    print(f"Failed to search for artist. Status code: {response.status_code}")
    print(response.text)


Response Data: {'artists': {'href': 'https://api.spotify.com/v1/search?query=Rainbow+Kitten+Surprise&type=artist&offset=0&limit=1', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4hz8tIajF2INpgM0qzPJz2'}, 'followers': {'href': None, 'total': 1185758}, 'genres': ['pov: indie'], 'href': 'https://api.spotify.com/v1/artists/4hz8tIajF2INpgM0qzPJz2', 'id': '4hz8tIajF2INpgM0qzPJz2', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5eb88aed23707da4abd1b8e2fe7', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab6761610000517488aed23707da4abd1b8e2fe7', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f17888aed23707da4abd1b8e2fe7', 'width': 160}], 'name': 'Rainbow Kitten Surprise', 'popularity': 71, 'type': 'artist', 'uri': 'spotify:artist:4hz8tIajF2INpgM0qzPJz2'}], 'limit': 1, 'next': 'https://api.spotify.com/v1/search?query=Rainbow+Kitten+Surprise&type=artist&offset=1&limit=1', 'offset': 0, 'previous': Non

In [24]:
# Step 6: Get artist's top tracks
# Define the endpoint for the artist's top tracks
top_tracks_url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
params = {
    "market": "US"  # Specify the market (country)
}

# Make the API request to get top tracks
response = requests.get(top_tracks_url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    top_tracks_data = response.json()
    print(f"Top tracks for {artist_name}:")
    for idx, track in enumerate(top_tracks_data['tracks'], start=1):
        print(f"Track data: {track}")
        track_name = track['name']
        album_name = track['album']['name']
        duration_ms = track['duration_ms']
        popularity = track['popularity']
        # print(f"{idx}. {track_name} - Album: {album_name}")
else:
    print(f"Failed to get top tracks. Status code: {response.status_code}")
    print(response.text)

Top tracks for Rainbow Kitten Surprise:
Track data: {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4hz8tIajF2INpgM0qzPJz2'}, 'href': 'https://api.spotify.com/v1/artists/4hz8tIajF2INpgM0qzPJz2', 'id': '4hz8tIajF2INpgM0qzPJz2', 'name': 'Rainbow Kitten Surprise', 'type': 'artist', 'uri': 'spotify:artist:4hz8tIajF2INpgM0qzPJz2'}], 'external_urls': {'spotify': 'https://open.spotify.com/album/7vXiklrUeffx5o2F8VYwm3'}, 'href': 'https://api.spotify.com/v1/albums/7vXiklrUeffx5o2F8VYwm3', 'id': '7vXiklrUeffx5o2F8VYwm3', 'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b2739529217e48fc8bab89c8a1f9', 'width': 640, 'height': 640}, {'url': 'https://i.scdn.co/image/ab67616d00001e029529217e48fc8bab89c8a1f9', 'width': 300, 'height': 300}, {'url': 'https://i.scdn.co/image/ab67616d000048519529217e48fc8bab89c8a1f9', 'width': 64, 'height': 64}], 'is_playable': True, 'name': 'How to: Friend, Love, Freefall', 'release_date': '2018-04-06', '

In [22]:
# Step 7: Transform to pandas df
import pandas as pd

tracks_df = pd.DataFrame.from_records(top_tracks_data['tracks'])
tracks_df.sort_values(["popularity"], inplace = True)

print(tracks_df.head(3))


                                               album  \
6  {'album_type': 'album', 'artists': [{'external...   
7  {'album_type': 'album', 'artists': [{'external...   
9  {'album_type': 'album', 'artists': [{'external...   

                                             artists  disc_number  \
6  [{'external_urls': {'spotify': 'https://open.s...            1   
7  [{'external_urls': {'spotify': 'https://open.s...            1   
9  [{'external_urls': {'spotify': 'https://open.s...            1   

   duration_ms  explicit              external_ids  \
6       173610     False  {'isrc': 'USAT22400510'}   
7       206626     False  {'isrc': 'QMAAK1557437'}   
9       231906     False  {'isrc': 'QMAAK1557436'}   

                                       external_urls  \
6  {'spotify': 'https://open.spotify.com/track/66...   
7  {'spotify': 'https://open.spotify.com/track/0w...   
9  {'spotify': 'https://open.spotify.com/track/0M...   

                                                href    